In [23]:
# Cell 1: imports & seed
import csv
from datetime import datetime
import random, math, time
from copy import deepcopy
import matplotlib.pyplot as plt

random.seed(42)


In [24]:
# Cell 2: data loader with normalization
def load_csv(path):
    with open(path, 'r') as f:
        reader = csv.DictReader(f)
        data = [row for row in reader]
    return reader.fieldnames, data

def merge_and_sort(normal_csv, attack_csv, augment_devices=5):
    h1, normal = load_csv(normal_csv)
    h2, attack = load_csv(attack_csv)
    for r in normal: r['label'] = 0
    for r in attack: r['label'] = 1
    combined = normal + attack
    # Robust timestamp parsing: try several formats
    for i, row in enumerate(combined):
        d = row.get('Date','').strip()
        t = row.get('Time','').strip()
        parsed = None
        for fmt in ("%d-%b-%y, %H:%M:%S", "%d-%b-%y %H:%M:%S", "%Y-%m-%d %H:%M:%S", "%d/%m/%Y %H:%M:%S"):
            try:
                parsed = datetime.strptime(f"{d} {t}", fmt)
                break
            except Exception:
                continue
        if parsed is None:
            # fallback: try splitting commas
            try:
                parsed = datetime.strptime((d + " " + t).replace(',', ' '), "%d %b %y %H:%M:%S")
            except Exception:
                parsed = datetime(1970,1,1,0,0,0)
        row['timestamp'] = parsed
        # device id fallback
        if not row.get('device_id'):
            row['device_id'] = 'device_' + str((i % augment_devices) + 1)
        # Fridge_Temperature normalization: scale to ~0-1 by dividing 40 (safe for fridge ranges)
        ft = row.get('Fridge_Temperature', '').strip()
        try:
            val = float(ft)
        except Exception:
            try:
                val = float(ft.replace(',','.'))
            except Exception:
                val = 0.0
        row['Fridge_Temperature_norm'] = val / 40.0   # tuned normalization
        tc = row.get('Temp_Condition','').strip().lower()
        row['Temp_Condition_enc'] = 1.0 if 'high' in tc else 0.0
    combined.sort(key=lambda x: x['timestamp'])
    return combined

def build_time_windows(parsed, window_size=5, step=1):
    # Build sliding windows across time; windows are lists of sequential rows
    windows = []
    for i in range(0, max(1, len(parsed)-window_size+1), step):
        seq_rows = parsed[i:i+window_size]
        X_seq = []
        lbl = 0
        for r in seq_rows:
            X_seq.append([r['Fridge_Temperature_norm'], r['Temp_Condition_enc']])
            lbl = max(lbl, int(r['label']))
        windows.append({'X_seq': X_seq, 'label': lbl, 'timestamp': seq_rows[-1]['timestamp']})
    return windows

# Quick run to ensure loader works (paths must be present in notebook)
# headerless test example usage omitted here - run later after uploading CSVs.


In [25]:
# Cell 3: improved TGNN class with momentum and input-weight updates
class TGNN:
    def __init__(self, input_dim=2, hidden_dim=16, seed=0):
        rnd = random.Random(seed)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        # weight matrices
        self.W_in = [[rnd.uniform(-0.2,0.2) for _ in range(input_dim)] for __ in range(hidden_dim)]
        self.W_h  = [[rnd.uniform(-0.2,0.2) for _ in range(hidden_dim)] for __ in range(hidden_dim)]
        # output layer (trainable)
        self.W_out = [rnd.uniform(-0.1,0.1) for _ in range(hidden_dim)]
        self.b_out = 0.0
        # momentum terms
        self.v_out = [0.0]*hidden_dim
        self.v_in = [[0.0]*input_dim for _ in range(hidden_dim)]

    @staticmethod
    def relu_vec(v): return [x if x>0 else 0.0 for x in v]
    @staticmethod
    def sigmoid(x):
        # numeric-stable
        if x >= 0:
            z = math.exp(-x)
            return 1.0/(1.0+z)
        else:
            z = math.exp(x)
            return z/(1.0+z)

    def matvec(self, M, v):
        return [sum(M[i][j]*v[j] for j in range(len(v))) for i in range(len(M))]

    def forward_full(self, X_seq):
        # returns final hidden states H (list of vectors) and final h
        h = [0.0]*self.hidden_dim
        H_seq = []
        for x in X_seq:
            in_part = self.matvec(self.W_in, x)
            hid_part = self.matvec(self.W_h, h)
            h = self.relu_vec([in_part[i] + hid_part[i] for i in range(self.hidden_dim)])
            H_seq.append(h[:])
        return H_seq, h

    def predict(self, X_seq):
        _, h_final = self.forward_full(X_seq)
        s = sum(h_final[i]*self.W_out[i] for i in range(self.hidden_dim)) + self.b_out
        return self.sigmoid(s)

    def train(self, windows, epochs=5, lr_out=0.05, lr_in=0.005, batch_size=32, momentum=0.9):
        n = len(windows)
        if n == 0:
            return
        for ep in range(1, epochs+1):
            random.shuffle(windows)
            total_loss = 0.0
            for start in range(0, n, batch_size):
                batch = windows[start:start+batch_size]
                # accumulate grads
                grad_w_out = [0.0]*self.hidden_dim
                grad_b = 0.0
                grad_W_in = [[0.0]*self.input_dim for _ in range(self.hidden_dim)]
                for win in batch:
                    X = win['X_seq']
                    y = float(win['label'])
                    H_seq, h_final = self.forward_full(X)
                    p = self.predict(X)
                    p = min(max(p, 1e-6), 1-1e-6)
                    loss = - (y*math.log(p) + (1-y)*math.log(1-p))
                    total_loss += loss
                    # gradient scalar for output pre-sigmoid (dL/ds)
                    grad_scalar = (p - y)
                    # grad w.r.t W_out = grad_scalar * h_final
                    for k in range(self.hidden_dim):
                        grad_w_out[k] += grad_scalar * h_final[k]
                    grad_b += grad_scalar
                    # small gradient back to input weights (heuristic)
                    # we propagate grad_scalar uniformly to W_in (not full backprop through nonlinearities)
                    for i in range(self.hidden_dim):
                        for j in range(self.input_dim):
                            # scale by h_final to prioritize active neurons
                            grad_W_in[i][j] += grad_scalar * (h_final[i] * 0.01) * X[-1][j]
                # average grads
                m = len(batch)
                for k in range(self.hidden_dim):
                    g = grad_w_out[k] / m
                    # momentum update
                    self.v_out[k] = momentum * self.v_out[k] + lr_out * g
                    self.W_out[k] -= self.v_out[k]
                self.b_out -= (grad_b / m) * lr_out
                for i in range(self.hidden_dim):
                    for j in range(self.input_dim):
                        g_in = (grad_W_in[i][j] / m)
                        self.v_in[i][j] = momentum * self.v_in[i][j] + lr_in * g_in
                        self.W_in[i][j] -= self.v_in[i][j]
            avg_loss = total_loss / n
            print(f"Epoch {ep} | Loss={avg_loss:.6f}")


In [26]:
# Cell 4: PPO agent improvements (epsilon exploration and stable update)
class PPOAgent:
    def __init__(self, state_dim=3, action_dim=4, seed=0):
        rnd = random.Random(seed)
        self.state_dim = state_dim
        self.action_dim = action_dim
        # actor weights: action_dim x state_dim
        self.actor_w = [[rnd.uniform(-0.1,0.1) for _ in range(state_dim)] for _ in range(action_dim)]
        self.actor_b = [0.0]*action_dim
        # critic linear
        self.critic_w = [rnd.uniform(-0.1,0.1) for _ in range(state_dim)]
        self.critic_b = 0.0
        # buffer
        self.reset_buffer()
        self.epsilon = 0.1  # exploration prob

    def softmax(self, logits):
        maxv = max(logits)
        ex = [math.exp(l - maxv) for l in logits]
        s = sum(ex)
        return [e/s for e in ex]

    def policy(self, state):
        logits = [sum(self.actor_w[a][i]*state[i] for i in range(self.state_dim)) + self.actor_b[a] for a in range(self.action_dim)]
        return self.softmax(logits)

    def value(self, state):
        return sum(self.critic_w[i]*state[i] for i in range(self.state_dim)) + self.critic_b

    def select_action(self, state):
        # epsilon-greedy style exploration mixed with softmax sampling
        if random.random() < self.epsilon:
            a = random.randrange(self.action_dim)
            probs = [1.0/self.action_dim]*self.action_dim
            return a, probs
        probs = self.policy(state)
        # categorical sample
        r = random.random()
        cum = 0.0
        a = 0
        for i,p in enumerate(probs):
            cum += p
            if r <= cum:
                a = i
                break
        return a, probs

    def store_transition(self, s, a, r, v):
        self.buf_states.append(s)
        self.buf_actions.append(a)
        self.buf_rewards.append(r)
        self.buf_values.append(v)

    def reset_buffer(self):
        self.buf_states = []
        self.buf_actions = []
        self.buf_rewards = []
        self.buf_values = []

    def compute_returns_advs(self, gamma=0.99):
        R = []
        G = 0.0
        for r in reversed(self.buf_rewards):
            G = r + gamma * G
            R.insert(0, G)
        advs = [R[i] - self.buf_values[i] for i in range(len(R))]
        # normalize advs
        if len(advs)>0:
            mean = sum(advs)/len(advs)
            var = sum((x-mean)**2 for x in advs)/len(advs)
            std = math.sqrt(var) if var>0 else 1.0
            advs = [(a-mean)/ (std + 1e-8) for a in advs]
        return R, advs

    def update(self, lr_actor=0.01, lr_critic=0.01, clip=0.2, epochs=3):
        if not self.buf_states:
            return
        R, advs = self.compute_returns_advs()
        for _ in range(epochs):
            for i, s in enumerate(self.buf_states):
                a = self.buf_actions[i]
                adv = advs[i]
                # update actor (policy gradient approx)
                probs = self.policy(s)
                one_hot = [0.0]*self.action_dim; one_hot[a]=1.0
                for j in range(self.action_dim):
                    grad_coeff = (one_hot[j] - probs[j]) * adv
                    for k in range(self.state_dim):
                        self.actor_w[j][k] += lr_actor * grad_coeff * s[k]
                    self.actor_b[j] += lr_actor * grad_coeff
                # critic update (MSE)
                td = R[i] - self.buf_values[i]
                for k in range(self.state_dim):
                    self.critic_w[k] += lr_critic * td * s[k]
                self.critic_b += lr_critic * td
        self.reset_buffer()


In [27]:
# Cell 5: honeypot
class Honeypot:
    def __init__(self):
        self.captured = []
    def attract(self, win):
        self.captured.append(win)
    def get_captured(self):
        return len(self.captured)


In [28]:
# Cell 6: helpers
def extract_state(win_score, win):
    # state: [win_score, avg_temp_norm, avg_cond]
    avg_temp = sum(x[0] for x in win['X_seq']) / len(win['X_seq'])
    avg_cond = sum(x[1] for x in win['X_seq']) / len(win['X_seq'])
    # normalize avg_temp to small number (already normalized)
    return [min(1.0, abs(win_score)), avg_temp, avg_cond]

def evaluate_model(model, windows):
    tp=tn=fp=fn=0
    for w in windows:
        p = model.predict(w['X_seq'])
        pred = 1 if p>0.5 else 0
        t = w['label']
        if t==1 and pred==1: tp+=1
        elif t==0 and pred==0: tn+=1
        elif t==0 and pred==1: fp+=1
        elif t==1 and pred==0: fn+=1
    total = tp+tn+fp+fn if (tp+tn+fp+fn)>0 else 1
    acc = (tp+tn)/total
    prec = tp/(tp+fp+1e-8)
    rec = tp/(tp+fn+1e-8)
    f1 = 2*prec*rec/(prec+rec+1e-8)
    return {'tp':tp,'tn':tn,'fp':fp,'fn':fn,'acc':acc,'prec':prec,'rec':rec,'f1':f1}

def plot_rounds(accs, fname='federated_accuracy.png'):
    plt.figure(figsize=(7,4))
    plt.plot(range(1,len(accs)+1), accs, marker='o', color='teal')
    plt.title('Federated TGNN Accuracy per Round')
    plt.xlabel('Round'); plt.ylabel('Accuracy'); plt.grid(True)
    plt.savefig(fname, dpi=150)
    plt.show()


In [29]:
# Cell 7: federated training loop (main)
# Configuration - tune for Colab GPU runtime if you switch to vectorized versions later
NORMAL_CSV = 'normal_data.csv'
ATTACK_CSV = 'attack_data.csv'
CLIENTS = 3
FED_ROUNDS = 6
WINDOW_SIZE = 5
TGNN_EPOCHS = 6        # per-client local epochs
TGNN_BATCH = 64
PPO_ITERS = 20         # how many decisions per client per round
LR_OUT = 0.08
LR_IN  = 0.01

print("Loading data...")
parsed = merge_and_sort(NORMAL_CSV, ATTACK_CSV, augment_devices=10)
windows = build_time_windows(parsed, window_size=WINDOW_SIZE, step=1)
print("Total windows:", len(windows))

# partition windows to clients (round-robin)
client_windows = [[] for _ in range(CLIENTS)]
for idx, w in enumerate(windows):
    client_windows[idx % CLIENTS].append(w)

# initialize models
clients_tgnn = [TGNN(input_dim=2, hidden_dim=16, seed=100+i) for i in range(CLIENTS)]
clients_ppo  = [PPOAgent(state_dim=3, action_dim=4, seed=200+i) for i in range(CLIENTS)]
clients_hp   = [Honeypot() for _ in range(CLIENTS)]

global_tgnn = deepcopy(clients_tgnn[0])
global_ppo  = deepcopy(clients_ppo[0])

acc_rounds = []

for rnd in range(1, FED_ROUNDS+1):
    print("\n=== Federated Round %d/%d ===" % (rnd, FED_ROUNDS))
    local_tgnn_models = []
    local_ppo_models  = []
    round_start = time.time()
    for cid in range(CLIENTS):
        data = client_windows[cid]
        if not data:
            local_tgnn_models.append(clients_tgnn[cid])
            local_ppo_models.append(clients_ppo[cid])
            continue
        # local TGNN training
        print(f" Client {cid+1}: TGNN training on {len(data)} windows...")
        clients_tgnn[cid].train(data, epochs=TGNN_EPOCHS, lr_out=LR_OUT, lr_in=LR_IN, batch_size=TGNN_BATCH, momentum=0.9)

        # PPO interactions: sample windows and let PPO act with TGNN score
        for it in range(PPO_ITERS):
            w = random.choice(data)
            p = clients_tgnn[cid].predict(w['X_seq'])
            state = extract_state(p, w)
            action, probs = clients_ppo[cid].select_action(state)
            # reward rules
            if w['label'] == 1:
                if action == 2:
                    clients_hp[cid].attract(w)
                    reward = 6.0
                elif action == 1:
                    reward = 3.0
                elif action == 0:
                    reward = -3.0
                else:
                    reward = 1.0
            else:
                # normal traffic
                if action == 1:   # false quarantine
                    reward = -2.0
                else:
                    reward = 0.5
            value = clients_ppo[cid].value(state)
            clients_ppo[cid].store_transition(state, action, reward, value)
        # update PPO locally
        clients_ppo[cid].update(lr_actor=0.01, lr_critic=0.01, epochs=4)
        local_tgnn_models.append(clients_tgnn[cid])
        local_ppo_models.append(clients_ppo[cid])
    # Server aggregation (FedAvg)
    # Average W_out and b_out across local tgnns
    n = len(local_tgnn_models)
    if n>0:
        # average W_out
        for k in range(global_tgnn.hidden_dim):
            global_tgnn.W_out[k] = sum(m.W_out[k] for m in local_tgnn_models)/n
        global_tgnn.b_out = sum(m.b_out for m in local_tgnn_models)/n
    # average actor weights for PPO
    npp = len(local_ppo_models)
    if npp>0:
        for a in range(global_ppo.action_dim):
            for k in range(global_ppo.state_dim):
                global_ppo.actor_w[a][k] = sum(m.actor_w[a][k] for m in local_ppo_models)/npp
            global_ppo.actor_b[a] = sum(m.actor_b[a] for m in local_ppo_models)/npp
        for k in range(global_ppo.state_dim):
            global_ppo.critic_w[k] = sum(m.critic_w[k] for m in local_ppo_models)/npp
        global_ppo.critic_b = sum(m.critic_b for m in local_ppo_models)/npp

    # distribute global back to clients (only output layer / actor/critic)
    for cid in range(CLIENTS):
        clients_tgnn[cid].W_out = global_tgnn.W_out[:]
        clients_tgnn[cid].b_out = global_tgnn.b_out
        clients_ppo[cid].actor_w = deepcopy(global_ppo.actor_w)
        clients_ppo[cid].actor_b = deepcopy(global_ppo.actor_b)
        clients_ppo[cid].critic_w = deepcopy(global_ppo.critic_w)
        clients_ppo[cid].critic_b = global_ppo.critic_b

    # evaluate aggregated model on full windows
    metrics = evaluate_model(global_tgnn, windows)
    acc_rounds.append(metrics['acc'])
    print("Round %d done in %.1f s: acc=%.4f, prec=%.4f, rec=%.4f, f1=%.4f" %
          (rnd, time.time()-round_start, metrics['acc'], metrics['prec'], metrics['rec'], metrics['f1']))

# final evaluation & plots
final_metrics = evaluate_model(global_tgnn, windows)
print("\n=== Final evaluation ===")
print(final_metrics)
plot_rounds(acc_rounds)
print("Total honeypot captures per client:", [hp.get_captured() for hp in clients_hp])


Loading data...
Total windows: 70381

=== Federated Round 1/6 ===
 Client 1: TGNN training on 23461 windows...
Epoch 1 | Loss=0.684279
Epoch 2 | Loss=0.683789
Epoch 3 | Loss=0.683732
Epoch 4 | Loss=0.683706
Epoch 5 | Loss=0.683689
Epoch 6 | Loss=0.683746
 Client 2: TGNN training on 23460 windows...
Epoch 1 | Loss=0.684372
Epoch 2 | Loss=0.683698
Epoch 3 | Loss=0.683705
Epoch 4 | Loss=0.683694
Epoch 5 | Loss=0.683686
Epoch 6 | Loss=0.683659
 Client 3: TGNN training on 23460 windows...
Epoch 1 | Loss=0.684222
Epoch 2 | Loss=0.683745
Epoch 3 | Loss=0.683730
Epoch 4 | Loss=0.683638
Epoch 5 | Loss=0.683648
Epoch 6 | Loss=0.683666
Round 1 done in 206.4 s: acc=0.5691, prec=0.0000, rec=0.0000, f1=0.0000

=== Federated Round 2/6 ===
 Client 1: TGNN training on 23461 windows...
Epoch 1 | Loss=0.683709
Epoch 2 | Loss=0.683797
Epoch 3 | Loss=0.683774
Epoch 4 | Loss=0.683720
Epoch 5 | Loss=0.683724
Epoch 6 | Loss=0.683692
 Client 2: TGNN training on 23460 windows...
Epoch 1 | Loss=0.683646
Epoch 2 

KeyboardInterrupt: 